#Earthquake time-forecasting CNN

**mount Google Drive as a disk to access files and data**






In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd "/content/drive/MyDrive/Dissertation/Prototype 4" #Change the directory name according to your convenience and where you have saved all the data files

In [ ]:
import pickle
import numpy as np

**set choice for input files (determines the type of normalisation)**

In [ ]:
choice = 'new2' # old=norm on all 3 components together. new2=no norm. new=norm on individual componenets, 


**set name of output model weights**

In [ ]:
# name of model weights file with version number of the training
tnum=1
WeightsFile='./model_'+str(tnum)+'.h5'
print('weights for the pending training will be saved in:',WeightsFile)

In [ ]:
#WeightsFile='./model_0'+'.h5'

In [ ]:
#WeightsFile='./bestweights'+'.h5'

**read input files**

In [ ]:
normnoise = pickle.load( open( "./Data/normnoise_"+choice+".p", "rb" ) )
normpre = pickle.load( open( "./Data/normpre_"+choice+".p", "rb" ) )
testlist = pickle.load( open( "./Data/testlist.p", "rb" ) )
trainlist = pickle.load( open( "./Data/trainlist.p", "rb" ) )

In [ ]:
print(len(normpre))
print(len(trainlist))
print(len(testlist))

In [ ]:
max(trainlist)

In [ ]:
x_train = [normpre[i] for i in trainlist]
x_test = [normpre[i] for i in testlist]
x_noise_train = [normnoise[i] for i in trainlist]
x_noise_test = [normnoise[i] for i in testlist]

window_length=16384

#Make data into a format such that it can be concatenated
x_train2 = np.reshape(x_train, (len(x_train)*len(x_train[0]), window_length, 3))
x_test2 = np.reshape(x_test, (len(x_test)*len(x_test[0]), window_length, 3))
x_noise_train2 = np.expand_dims(x_noise_train, axis=-1)
x_noise_test2 = np.expand_dims(x_noise_test, axis=-1)
x_noise_train2 = np.reshape(x_noise_train2, (len(x_noise_train)*len(x_noise_train[0]), window_length, 3))
x_noise_test2 = np.reshape(x_noise_test2, (len(x_noise_test)*len(x_noise_test[0]), window_length, 3))

# Generate ground truth (ones = 'precursor' class, zeros = 'noise' class)
y_train2 = np.ones(len(x_train2))
y_test2 =np.ones(len(x_test2))
y_noise_train2 = np.zeros(len(x_noise_train2))
y_noise_test2 = np.zeros(len(x_noise_test2))
y_train2  = y_train2.tolist()
y_test2 = y_test2.tolist()
y_noise_train2 = y_noise_train2.tolist()
y_noise_test2 = y_noise_test2.tolist()

#Concatenate noise and precursor datasets 
x_train = np.append(x_train2,x_noise_train2, axis=0)
x_test = np.append(x_test2,x_noise_test2, axis=0)
y_train = np.append(y_train2,y_noise_train2, axis=0)
y_test = np.append(y_test2,y_noise_test2, axis=0)

**Build the CNN Footprint**

In [ ]:

!pip install pip install np_utils

#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
y_train= to_categorical(y_train, num_classes=2)
y_test= to_categorical(y_test, num_classes=2)

import tensorflow as tf 
import numpy as np
import time
import os
from keras.models import Model
from keras.layers import Conv1D, BatchNormalization, Add, MaxPooling1D, Dropout, Dense, Flatten, CuDNNLSTM, ZeroPadding1D
from keras.layers import Activation, Input, concatenate, GaussianNoise, GlobalMaxPooling1D, GlobalAveragePooling1D, Softmax, Permute, Multiply, Masking
#from keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.optimizers import Adam,RMSprop, SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.regularizers import *  

def encoder(input_shape, nb_classes):
    
    input_layer= Input(input_shape)
    random = Conv1D(3, 3, padding='same', kernel_initializer='random_normal')(input_layer)

    X = Add()([input_layer, random])

    conv3 = Conv1D(filters=32, kernel_size=7,padding='same', strides=1, kernel_initializer='random_normal')(X)
    conv3 = Conv1D(filters=32, kernel_size=7,padding='same', strides=1, kernel_initializer='random_normal')(conv3)
    conv3 = MaxPooling1D(3, strides=1)(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)

    conv4 = Conv1D(64, kernel_size=7,padding='same', strides=1, kernel_initializer='random_normal')(conv3)
    conv4 = Conv1D(64, kernel_size=7,padding='same', strides=1, kernel_initializer='random_normal')(conv4)        
    conv4 = MaxPooling1D(3, strides=2)(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)

    conv5 = Conv1D(filters=64, kernel_size=5, padding='same', dilation_rate=2, kernel_initializer='random_normal')(conv4)
    conv5 = MaxPooling1D(3, strides=1)(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
        
    conv3 = Conv1D(filters=128, kernel_size=5,padding='same', dilation_rate=4 , kernel_initializer='random_normal')(conv5)
    conv3 = MaxPooling1D(3, strides=1)(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)

    conv3 = Conv1D(filters=128, kernel_size=5,padding='same', dilation_rate=8 , kernel_initializer='random_normal')(conv3)
    conv3 = MaxPooling1D(3, strides=1)(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)

    conv4 = Conv1D(256, kernel_size=3,padding='same',  dilation_rate=16 , kernel_initializer='random_normal')(conv3)
    conv4 = MaxPooling1D(3, strides=1)(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)

    conv5 = Conv1D(filters=256, kernel_size=3, padding='same', dilation_rate=32, kernel_initializer='random_normal')(conv4)
    conv5 = MaxPooling1D(3, strides=1)(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Dropout(0.02)(conv5)
  
    Y = GlobalAveragePooling1D()(conv5)
    Y = Dense(256, activation='relu', kernel_initializer='random_normal')(Y)
    Y = Dropout(0.02)(Y)

  
    output_layer = Dense(nb_classes, activation='softmax', kernel_initializer='random_normal')(Y)
    
    model = Model(inputs=input_layer, outputs=output_layer)

        
    return model      
    
model=encoder((window_length, 3), 2)
model.compile(loss='binary_crossentropy', optimizer=RMSprop(0.0001), metrics=['accuracy'])
model.summary()  


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
from matplotlib import pyplot as plt
#from keras.callbacks import TensorBoard
#from keras import backend as K


class My_Callback(Callback):

    def on_epoch_begin(self, epoch, logs={}):
        session = K.get_session()
        self.model.layers[1].kernel.initializer.run(session=session)
        return

    def on_epoch_end(self, epoch, logs={}):
        #layer_index = 0  ## index of the layer you want to change
        # random weights to reset the layer
        #new_weights = np.random.randn(*self.model.layers[layer_index].get_weights().shape)

        #self.model.layers[layer_index].set_weights(new_weights)

        return

callbacks = [
    EarlyStopping(patience=30, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.000001, verbose=1),
    ModelCheckpoint(filepath=WeightsFile, verbose=1, save_best_only=True, monitor='val_accuracy', save_weights_only=False, mode='max'),
    My_Callback()]



**Start training the CNN**

In [ ]:
#!%%pypy
# This callback reinitialises the weights in the first layer of the neural network (random layer in my report) at the start of each epoch.
class My_Callback(Callback):

    def on_epoch_begin(self, epoch, logs={}):
        # Code change to implement random layer in new version of TensorFlow
        # Get current weights
        weights = self.model.layers[1].get_weights()
        new_weights = []
        # Only the kernel is re-initialized, not the bais. For this reason I copy the old bais in the new weights.
        # I want the new weights to have the same distribution as the old ones. 
        # For this reason, I scale the normal distribution by the stddev and mean of the initializer of the layer.
        # To make sure that the right number of weights are generated, I use the shape of the old weights as a template.
        new_weights.append(self.model.layers[1].kernel_initializer.stddev * np.random.randn(*weights[0].shape) + self.model.layers[1].kernel_initializer.mean)
        
        new_weights.append(weights[1])
        self.model.layers[1].set_weights(new_weights)
        
        return

    def on_epoch_end(self, epoch, logs={}):

        return

# Changed val_acc into val_accuracy in monitor to fir with new version of TensorFlow

# Changed Early Stopping patience from 15 to 30 (old)
# Changed Early Stopping patience from 15 to 120 (new)

callbacks = [
    EarlyStopping(patience=120, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    #ModelCheckpoint('weights_nonorm.h5', verbose=1, save_best_only=True, monitor='val_accuracy', save_weights_only=True),
    ModelCheckpoint(WeightsFile, verbose=1, save_best_only=True, monitor='val_accuracy', save_weights_only=True),

My_Callback()]

#model.fit(x_train, y_train, batch_size=16, epochs=100, shuffle=True, callbacks=callbacks,
#                    validation_data=(x_test, y_test))
history = model.fit(x_train, y_train, batch_size=32, epochs=100, shuffle=True, callbacks=callbacks, validation_data=(x_test, y_test))

**Loading the saved weights and testing the network on both train and test datasets:**

In [ ]:
model.load_weights(WeightsFile)
Tr = model.evaluate(x_train, y_train, verbose=1)[-1]
Te = model.evaluate(x_test, y_test, verbose=1)[-1]

print('Train Accuracy: ', Tr)
print('Test Accuracy: ', Te)

In [ ]:
#Training and validation loss

loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1,101)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#Training and validation accuracy

loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
epochs = range(1,101)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
""" import torch 
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as trans

dataloader = torch.utils.data.DataLoader(x_train, batch_size=32) """

In [ ]:
""" from sklearn.metrics import confusion_matrix

# simply call the confusion_matrix function to build a confusion matrix
cm = confusion_matrix(x_train, train_preds.argmax(dim=1))
print(cm) """

In [ ]:
""" #Build a Confusion Matrix



# get all the predictions for the entire training set
import torch
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    #model.eval() # set model to evaluate mode
    
    #for images, labels in loader:
        #images = images.to(device)
        #labels = labels.to(device)
        #preds = model(images)
        #all_preds = torch.cat((all_preds, preds), dim=0)
    
    return all_preds 

with torch.no_grad(): # disable gradient computations 
    train_preds = get_all_preds(model, dataloader)
    # print(train_preds.shape) # (6000,10) 
    # print(train_preds.requires_grad) # False

print('true labels: ', x_train.targets.numpy())
print('pred labels: ', y_train.argmax(dim=1).numpy()) """

In [ ]:
""" #Plotting the confusion matrix

import matplotlib.pyplot as plt
import numpy as np
import itertools

def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=None, normalize=False):
    
    arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions
    
     
    if cmap is None:
        cmap = plt.get_cmap('Oranges')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylim(len(target_names)-0.5, -0.5)
    plt.ylabel('True labels')
    plt.xlabel('Predicted labels')
    plt.savefig(title + '.png', dpi=500, bbox_inches = 'tight')
    plt.show()
    
    
    
# a tuple for all the class names
target_names = ('Noise', 'Precursors')
plot_confusion_matrix(cm, target_names) """